# Setup

In [ ]:
from db_funcs import get_all_content, get_db_con
import dotenv
from utils import Sentence_Extractor
from neo4j_funcs import Neo4j_Driver
from EntityRelationshipExtractor import ER_Extractor
from tqdm import tqdm

dotenv.load_dotenv()

neo4j = Neo4j_Driver()
sentence_extractor = Sentence_Extractor()
er_extractor=ER_Extractor()

# Extract Sentances and Entity Relationships
For each content block from the database, extract all sentences from the content block. For each content block, extract all entities and relationships using the model and system defined in the entity relationship extractor. Finally, insert each extracted entity into the neo4j database keeping track of which content block the entity is from. Entities can be from multiple content blocks so neo4j stores an array of content ids for each entity node.

In [ ]:

with get_db_con().cursor() as cur:
    content_ids, all_content = get_all_content(cur)
for content_id, content in tqdm(zip(content_ids, all_content), total=len(content_ids), desc="Processing content"):
    sentences = sentence_extractor.get_sentences(content)
    triplets=er_extractor.extract_ERs(sentences)
    for e in triplets:
        neo4j.insert_relationship(e.head, "entity", e.type, e.tail, "entity", content_id)
    